# CREDIT CARD FRAUD DETECTION

In [26]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,KFold 
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report

# pd.set_option("display.max_rows",None)
import warnings
warnings.filterwarnings("ignore")

#### Loading the Dataset

In [27]:
df = pd.read_csv(r"C:\Users\ADMIN\Downloads\codsoft\creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [28]:
df.shape

(284807, 31)

#### Understanding and Pre-Processing the Data

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [30]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [31]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [32]:
x = df.drop("Class",axis=1)
y = df["Class"]

In [33]:
col = pd.MultiIndex.from_product([["Train","Test"],["Accuracy","F1 Score","Recall","Precision"]])
modelscore = pd.DataFrame(columns=col)
modelscore.index.name = "Model Name"

In [34]:
modelscore

Empty DataFrame
Columns: [(Train, Accuracy), (Train, F1 Score), (Train, Recall), (Train, Precision), (Test, Accuracy), (Test, F1 Score), (Test, Recall), (Test, Precision)]
Index: []

In [35]:
def model_cal(model_df,model_name,model,x,y):
    xtrain,xtest,ytrain,ytest= train_test_split(x,y,random_state=10,test_size=0.20)
    xtrain,xtest,ytrain,ytest
    model.fit(xtrain,ytrain)
    ypred_train = model.predict(xtrain)
    ypred_test = model.predict(xtest)
    train_acc = accuracy_score(ytrain,ypred_train)
    train_f1 = f1_score(ytrain,ypred_train)
    train_rec = recall_score(ytrain,ypred_train)
    train_pres = precision_score(ytrain,ypred_train)
    test_acc = accuracy_score(ytest,ypred_test)
    test_f1 = f1_score(ytest,ypred_test)
    test_rec = recall_score(ytest,ypred_test)
    test_pres = precision_score(ytest,ypred_test)
    model_df.loc[model_name,:]=[train_acc,train_f1,train_rec,train_pres,test_acc,test_f1,test_rec,test_pres]
    print(classification_report(ytrain,ypred_train))
    print(classification_report(ytest,ypred_test))
    return model_df

#### Logistic Regression

In [36]:
lr = LogisticRegression(random_state=10)
model_cal(modelscore,"Logistic Regression",lr,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       0.60      0.60      0.60       398

    accuracy                           1.00    227845
   macro avg       0.80      0.80      0.80    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       0.60      0.62      0.61        94

    accuracy                           1.00     56962
   macro avg       0.80      0.81      0.80     56962
weighted avg       1.00      1.00      1.00     56962



Train                                   Test           \
                     Accuracy F1 Score    Recall Precision  Accuracy F1 Score   
Model Name                                                                      
Logistic Regression  0.998613      0.6  0.595477  0.604592  0.998683  0.60733   

                                         
                       Recall Precision  
Model Name                               
Logistic Regression  0.617021  0.597938

#### Decision Tree

In [37]:
dt = DecisionTreeClassifier(random_state=10)
model_cal(modelscore,"Decision Tree",dt,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       1.00      1.00      1.00       398

    accuracy                           1.00    227845
   macro avg       1.00      1.00      1.00    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       0.80      0.78      0.79        94

    accuracy                           1.00     56962
   macro avg       0.90      0.89      0.89     56962
weighted avg       1.00      1.00      1.00     56962



Train                                   Test  \
                     Accuracy F1 Score    Recall Precision  Accuracy   
Model Name                                                             
Logistic Regression  0.998613      0.6  0.595477  0.604592  0.998683   
Decision Tree             1.0      1.0       1.0       1.0  0.999315   

                                                   
                     F1 Score    Recall Precision  
Model Name                                         
Logistic Regression   0.60733  0.617021  0.597938  
Decision Tree        0.789189  0.776596  0.802198

####  KNeighborsClassifier

In [38]:
knn = KNeighborsClassifier()
model_cal(modelscore,"KNN",knn,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       1.00      0.10      0.18       398

    accuracy                           1.00    227845
   macro avg       1.00      0.55      0.59    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       1.00      0.07      0.14        94

    accuracy                           1.00     56962
   macro avg       1.00      0.54      0.57     56962
weighted avg       1.00      1.00      1.00     56962



Train                                    Test  \
                     Accuracy  F1 Score    Recall Precision  Accuracy   
Model Name                                                              
Logistic Regression  0.998613       0.6  0.595477  0.604592  0.998683   
Decision Tree             1.0       1.0       1.0       1.0  0.999315   
KNN                  0.998429  0.182648  0.100503       1.0  0.998473   

                                                   
                     F1 Score    Recall Precision  
Model Name                                         
Logistic Regression   0.60733  0.617021  0.597938  
Decision Tree        0.789189  0.776596  0.802198  
KNN                  0.138614  0.074468       1.0

#### Random Forest

In [39]:
rf = RandomForestClassifier(random_state=10)
model_cal(modelscore,"Random Forest",rf,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       1.00      1.00      1.00       398

    accuracy                           1.00    227845
   macro avg       1.00      1.00      1.00    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       0.99      0.82      0.90        94

    accuracy                           1.00     56962
   macro avg       0.99      0.91      0.95     56962
weighted avg       1.00      1.00      1.00     56962



Train                                    Test  \
                     Accuracy  F1 Score    Recall Precision  Accuracy   
Model Name                                                              
Logistic Regression  0.998613       0.6  0.595477  0.604592  0.998683   
Decision Tree             1.0       1.0       1.0       1.0  0.999315   
KNN                  0.998429  0.182648  0.100503       1.0  0.998473   
Random Forest             1.0       1.0       1.0       1.0  0.999684   

                                                   
                     F1 Score    Recall Precision  
Model Name                                         
Logistic Regression   0.60733  0.617021  0.597938  
Decision Tree        0.789189  0.776596  0.802198  
KNN                  0.138614  0.074468       1.0  
Random Forest        0.895349  0.819149  0.987179

#### AdaBoost

In [40]:
adb = AdaBoostClassifier(random_state=10)
model_cal(modelscore,"AdaBoost",adb,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       0.81      0.76      0.78       398

    accuracy                           1.00    227845
   macro avg       0.90      0.88      0.89    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       0.82      0.78      0.80        94

    accuracy                           1.00     56962
   macro avg       0.91      0.89      0.90     56962
weighted avg       1.00      1.00      1.00     56962



Train                                    Test  \
                     Accuracy  F1 Score    Recall Precision  Accuracy   
Model Name                                                              
Logistic Regression  0.998613       0.6  0.595477  0.604592  0.998683   
Decision Tree             1.0       1.0       1.0       1.0  0.999315   
KNN                  0.998429  0.182648  0.100503       1.0  0.998473   
Random Forest             1.0       1.0       1.0       1.0  0.999684   
AdaBoost             0.999267  0.783398  0.758794  0.809651   0.99935   

                                                   
                     F1 Score    Recall Precision  
Model Name                                         
Logistic Regression   0.60733  0.617021  0.597938  
Decision Tree        0.789189  0.776596  0.802198  
KNN                  0.138614  0.074468       1.0  
Random Forest        0.895349  0.819149  0.987179  
AdaBoost             0.797814  0.776596  0.820225

#### Gradient Boosting

In [41]:
gdb = GradientBoostingClassifier(random_state=10)
model_cal(modelscore,"Gradient Boosting",gdb,x,y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227447
           1       0.81      0.84      0.83       398

    accuracy                           1.00    227845
   macro avg       0.91      0.92      0.91    227845
weighted avg       1.00      1.00      1.00    227845

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56868
           1       0.84      0.82      0.83        94

    accuracy                           1.00     56962
   macro avg       0.92      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962



Train                                    Test  \
                     Accuracy  F1 Score    Recall Precision  Accuracy   
Model Name                                                              
Logistic Regression  0.998613       0.6  0.595477  0.604592  0.998683   
Decision Tree             1.0       1.0       1.0       1.0  0.999315   
KNN                  0.998429  0.182648  0.100503       1.0  0.998473   
Random Forest             1.0       1.0       1.0       1.0  0.999684   
AdaBoost             0.999267  0.783398  0.758794  0.809651   0.99935   
Gradient Boosting    0.999381  0.826141  0.841709  0.811138  0.999438   

                                                   
                     F1 Score    Recall Precision  
Model Name                                         
Logistic Regression   0.60733  0.617021  0.597938  
Decision Tree        0.789189  0.776596  0.802198  
KNN                  0.138614  0.074468       1.0  
Random Forest        0.895349  0.819149  0.987179  
AdaBoost             0.797814  0.776596  0.820225  
Gradient Boosting    0.827957  0.819149  0.836957

In [42]:
acc = modelscore.iloc[:,0]-model_score.iloc[:,4]
acc[acc==min(acc)]

Model Name
AdaBoost   -0.000083
dtype: object

In [43]:
f1 = modelscore.iloc[:,1] - model_score.iloc[:,5]
f1[f1==min(f1)]

Model Name
AdaBoost   -0.014416
dtype: object

In [44]:
rec = modelscore.iloc[:,2]-model_score.iloc[:,6]
rec[rec==min(rec)]

Model Name
Logistic Regression   -0.021544
dtype: object

In [45]:
pres = modelscore.iloc[:,3]-model_score.iloc[:,6]
pres[pres==min(pres)]

Model Name
Logistic Regression   -0.012429
dtype: object

#### Logistic Regression gives a good results in both train and test dataset